The objective of this notebook is to:

* Load the dataset into the corresponding table in a database
* Execute the sql queries to gain insights about the original dataset

In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew

plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(10,6), dpi=180)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')
plt.tight_layout()

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv
/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


<Figure size 1800x1080 with 0 Axes>

In [51]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [53]:
import csv, sqlite3
import prettytable

prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data2.db")
cur = con.cursor()

In [54]:
%sql sqlite:///my_data2.db

In [55]:
train = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv', index_col='id')
org = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv')

In [56]:
org.to_sql('ACCIDENTTBL', con, if_exists='replace', index=False, method='multi')

10000

### Display five records of the Accident table

In [57]:
%sql select * from ACCIDENTTBL LIMIT 10;

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
rural,3,0.38,35,daylight,foggy,1,1,evening,1,1,1,0.2
highway,2,0.5,60,daylight,clear,1,0,afternoon,0,1,1,0.36
rural,2,0.95,60,daylight,clear,1,1,afternoon,1,0,0,0.52
rural,4,0.76,25,daylight,rainy,0,0,afternoon,0,1,2,0.34
highway,1,0.43,60,daylight,foggy,0,0,evening,1,1,4,0.53
highway,4,0.05,35,night,foggy,1,0,afternoon,0,1,3,0.39
rural,2,0.17,45,night,foggy,0,0,evening,0,1,1,0.28
urban,4,0.46,25,daylight,clear,1,1,morning,0,1,0,0.16
rural,4,0.81,60,dim,clear,1,0,morning,1,0,2,0.45
rural,2,0.38,45,dim,clear,1,1,morning,1,1,2,0.15


#### Display the names of unique <code>road_type</code> in the table

In [58]:
%sql select road_type from ACCIDENTTBL GROUP BY road_type;

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


road_type
highway
rural
urban


#### Display the average curvature for each <code>road_type</code>.

In [59]:
%sql select road_type, avg(curvature) as Average_Curvature from ACCIDENTTBL GROUP BY road_type;

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


road_type,Average_Curvature
highway,0.49872093023255903
rural,0.4961442861473174
urban,0.5024589184344196


#### Display the max accident risk where <code>lighting</code> was night.

In [60]:
%sql select lighting, max(accident_risk) as Max_Accident_Risk from ACCIDENTTBL WHERE Lighting = 'night';

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


lighting,Max_Accident_Risk
night,1.0


#### Display the weather, minimum and average accident risk where <code>weather</code> is foggy.

In [61]:
%sql select weather, min(accident_risk) as Min_Accident_Risk, avg(accident_risk) as Avg_Accident_Risk from ACCIDENTTBL WHERE weather = 'foggy';

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


weather,Min_Accident_Risk,Avg_Accident_Risk
foggy,0.0,0.4177710843373492


#### From the discussion below, author indicated that dataset had unrealistic entries such as highway with speed limit = 25 and zero past accidents with high accident risk. This could be due to noise generated intentionally or synthetic nature of the dataset where "labels may be generated deterministically from environmental and road conditions rather than historical accidents".

#### To confirm this claim, I have written some queries to get the count of the unrealistic entries.

#### Here is the link to the discussion: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/613636

In [48]:
%sql select COUNT(*) AS MATCH_COUNT from ACCIDENTTBL WHERE speed_limit = 25 AND road_type = 'highway';

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


MATCH_COUNT
686


In [49]:
%sql select COUNT(*) AS MATCH_COUNT from (select * from ACCIDENTTBL WHERE num_reported_accidents = 0 AND accident_risk > 0.8) as sub; 

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


MATCH_COUNT
13


In [62]:
%sql select COUNT(*) AS MATCH_COUNT from ACCIDENTTBL WHERE curvature = 1.0 AND road_type='urban';

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


MATCH_COUNT
15


#### The results from the queries, confirms the claim about the unrealistc entries in the synthetic dataset.

#### Writing the query to see whether <code>speed_limit</code>, <code>lighting</code>, <code>high curvature</code> and <code>weather</code> appears heavily influence accident risk

In [85]:
%sql select speed_limit, lighting, curvature, weather, accident_risk from ACCIDENTTBL ORDER BY accident_risk DESC LIMIT 20;

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


speed_limit,lighting,curvature,weather,accident_risk
60,night,0.84,rainy,1.0
70,night,0.92,rainy,1.0
70,night,0.87,rainy,0.97
70,night,0.98,foggy,0.96
70,night,0.99,rainy,0.94
70,night,0.89,rainy,0.94
70,night,0.93,rainy,0.93
60,night,0.86,foggy,0.92
70,night,0.99,rainy,0.92
70,night,0.75,rainy,0.92


#### We can see from the table that speed_limit, lighting, curvature, and weather does have heavy influence on the accident risk.

#### I am curious to see if lighting not equal to night and weather equal to clear leads to higher accident risk.

In [86]:
%sql select speed_limit, lighting, curvature, weather, accident_risk from ACCIDENTTBL WHERE lighting <> 'night' AND weather = 'clear' AND accident_risk>0.8;

   sqlite:///my_data1.db
 * sqlite:///my_data2.db
Done.


speed_limit,lighting,curvature,weather,accident_risk


#### Well the result is null, which these factors does not lead to higher accident risk.